In [3]:
import pandas as pd
import math
from Lilygo.Recording import Recording
from Lilygo.Dataset import Dataset
from os import listdir
from os.path import isfile, join
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
# feature extraction
from scipy import signal
from scipy.stats import entropy, kurtosis, skew
from scipy.signal import welch
from scipy.interpolate import interp1d
# to create a test and train split
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
# evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# Gradient boosting
import xgboost as xgb
from xgboost import XGBClassifier
# save and load models
import joblib

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

## Calculate features for training and testing

In [4]:
# This function aims to find the component caused by gravity from data, which means the signal around 0 Hz
def get_gravity(data):
    filtered_data = np.zeros_like(data)
    # Parameters in IIR filter
    alpla = [1, -1.979133761292768, 0.979521463540373]
    beta = [0.000086384997973502, 0.00012769995947004, 0.000086384997973502]
    # Formula of IIR filter
    for i in range(2, len(data)):
        filtered_data[i] = alpla[0] * (data[i] * beta[0] + data[i-1] * beta[1] + data[i-2] * beta[2] - filtered_data[i-1] * alpla[1] - filtered_data[i-2] * alpla[2])
    return filtered_data

# This function aims to realize a low-pass filter with cutoff frequency = 1 Hz. Because according to massive amounts of data, the general 
# minimum frequency of human walking is about 1 Hz
def get_highpass(data):
    filtered_data = np.zeros_like(data)  # filtered_data
    alpla = [1, -1.905384612118461, 0.910092542787947]
    beta = [0.953986986993339, -1.907503180919730, 0.953986986993339]
    for i in range(2, len(data)):
        filtered_data[i] = alpla[0] * (data[i] * beta[0] + data[i-1] * beta[1] + data[i-2] * beta[2] - filtered_data[i-1] * alpla[1] - filtered_data[i-2] * alpla[2])
    return filtered_data

# This funciton aims to realize a high-pass filter with cutoff frequency = 5 Hz. Because according to massive amounts of data, the general 
# maximum frequency of human walking is about 5 Hz
def get_lowpass(data):
    filtered_data = np.zeros_like(data)  # filtered_data
    alpla = [1, -1.80898117793047, 0.827224480562408]
    beta = [0.096665967120306, -0.172688631608676, 0.095465967120306]
    for i in range(2, len(data)):
        filtered_data[i] = alpla[0] * (data[i] * beta[0] + data[i-1] * beta[1] + data[i-2] * beta[2] - filtered_data[i-1] * alpla[1] - filtered_data[i-2] * alpla[2])
    return filtered_data

def pre_process(data):
    # Find the component caused by gravity from data and remove it from the singanl
    data_gravity = get_gravity(data)
    data_user = data - data_gravity
    # Get user's acceleration along the gravity direction by dot product
    data_acc = data_user * data_gravity
    # Add low pass and high pass filter to reduce noise in signal (possible human walking rate:1 - 5Hz)
    data_filtered = get_highpass(data_acc)
    data_filtered = get_lowpass(data_filtered)
    return data_filtered

def get_segment_overlap(data,sampling_rate, std_win):
    # Calculate raw magnitude of accelerometer signal
    # amagn_acc = [np.sqrt(a**2+trace.data['ay'].values[i]**2+trace.data['az'].values[i]**2)for i, a in enumerate(trace.data['ay'].values)]
    # Pre-process data
    data_seg = pre_process(data)
    # Calculate window size
    window_size = round(std_win*sampling_rate)
    segment_trace = [data_seg[s:s+window_size] for s in range(0, len(data_seg)-window_size, round(window_size/2))]
    return segment_trace

def get_segment_nonoverlap(data,sampling_rate, std_win):
    # Calculate window size
    window_size = round(std_win*sampling_rate)
    segment_trace = [data[s:s+window_size] for s in range(0, len(data)-window_size, window_size)]
    return segment_trace

### features for acticity

In [5]:
def extract_temporal_features(data):
    """
    returns time domain features (20) from windowed raw signal:
      - mean, standard deviation, kurtosis, skewness;
      - RMS, zero-crossing
      - The following percentiles: [0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100] (Q)
      - Range: max(x) - min(x)
    data: windowed signal
    """
    m = np.mean(data)
    sd = np.std(data)
    kurt = kurtosis(data)
    sk = skew(data)
    rms = np.sqrt(np.mean(data**2))
    zc = np.sum(np.diff(data>=m))
    q = np.array([0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1])
    Q = np.quantile(data, q)
    range = Q[-1]-Q[0]             
    return m, sd, kurt, sk, rms, zc, range, Q[0], Q[1], Q[2], Q[3], Q[4], Q[5], Q[6], Q[7], Q[8], Q[9], Q[10], Q[11], Q[12]


def extract_frequential_features(data, sr):
    """
    exctracting frequncy domain features (5) from windowed Fourier transform:
    - energy, entropy, centroid, bandwidth, max_freq
    data: windowed signal
    sf: sampling rate
    """
    window_size = len(data)
    data -= np.mean(data)
    ft = np.fft.fft(data)/window_size
    sr = int(sr)
    # get window length
    
    #discarding mirror part
    ft = ft[:window_size//2]
    #frequencies of the transofm
    freqs = np.fft.fftfreq(window_size, 1/sr)[1:window_size//2]
    #the spectral density is the squared of the absolute
    Spec = np.abs(ft)**2
    #Energy
    E = np.sum(Spec)/(window_size//2)
    #density
    P = Spec[1:]/np.sum(Spec[1:])
    #entropy
    H = -np.sum(P*np.log2(P))/np.log2((window_size//2))
    #centriod 
    C = np.sum(P*freqs)
    #Absolute distance  of frequencies from from Centroid
    distC = np.abs((C-freqs))
    #bandwidth is the weighted mean of the distance
    BW = np.sum(distC*P)
    #maximum frequency 
    max_fr = freqs[np.argmax(Spec[1:])]
    return E, H, C, BW, max_fr

# 25 features
def get_features_activity_win(data):
    # AC, DC = get_AC_DC(data, 200)
    m, sd, kurt, sk, rms, zc, range, q_0, q_1, q_2, q_3, q_4, q_5, q_6, q_7, q_8, q_9, q_10, q_11, q_12 = extract_temporal_features(data)
    E, H, C, BW, max_fr = extract_frequential_features(data, 200)
    all_features = [m, sd, kurt, sk, rms, zc, range, q_0, q_1, q_2, q_3, q_4, q_5, q_6, q_7, q_8, q_9, q_10, q_11, q_12, E, H, C, BW, max_fr]
    return all_features

def get_features_activity_trace(trace):
    # get segment data
    amagn = [np.sqrt(a**2+trace.data['ay'].values[i]**2+trace.data['az'].values[i]**2)for i, a in enumerate(trace.data['ax'].values)]
    std_win = 3 # length of window in seconds
    sampling_rate = 200
    segment_trace = get_segment_overlap(amagn, sampling_rate, std_win)
    # add feature extraction
    num_features = 25
    featured_trace_activity = np.zeros((np.shape(segment_trace)[0], num_features))
    for i in range(np.shape(segment_trace)[0]):
        featured_trace_activity[i,] = get_features_activity_win(segment_trace[i])
    return featured_trace_activity

### features for location

In [6]:
def get_feature_location(ax, ay, az, gx, gy, gz, sampling_rate = 200):
    acc_data = np.array([ax, ay, az]).T
    gyro_data = np.array([gx, gy, gz]).T
    # Compute magnitudes
    acc_magnitude = np.sqrt(np.sum(acc_data**2, axis=1))
    acc_mag_mean = abs(np.mean(acc_magnitude))
    acc_mag_std = np.std(acc_magnitude)


    # ----ACCELERATOR TIME DOMAIN----
    ax_mean = abs(np.mean(acc_data[:,0]))
    ay_mean = abs(np.mean(acc_data[:,1]))
    az_mean = abs(np.mean(acc_data[:,2]))
    a_mean_list = [ax_mean, ay_mean, az_mean]
    a_mean_list.sort() # Sorting list of numbers in ascending
    Am = a_mean_list[2] # Feature Am: the maximum mean among all dimensions (represents motion range for location)
    Bm = a_mean_list[2]/a_mean_list[1] # Feature Bm and Cm: ratio of the maximum mean in different axes (represents DoF in movement for location)
    Cm = a_mean_list[2]/a_mean_list[0] 

    ax_range = acc_data[np.argmax(acc_data[:,0]), 0] - acc_data[np.argmin(acc_data[:,0]), 0]
    ay_range = acc_data[np.argmax(acc_data[:,1]), 1] - acc_data[np.argmin(acc_data[:,1]), 1]
    az_range = acc_data[np.argmax(acc_data[:,2]), 2] - acc_data[np.argmin(acc_data[:,2]), 2]
    a_range_list = [ax_range, ay_range, az_range]
    a_range_list.sort() # Sorting list of numbers in ascending
    A = a_range_list[2] # Feature A: the maximum range among all dimensions (represents motion range for location)
    B = a_range_list[2]/a_range_list[1] # Feature B and C: ratio of the maximum ranges in different axes (represents DoF in movement for location)
    C = a_range_list[2]/a_range_list[0] 
    
    
    # ----GYROSCOPE TIME DOMAIN----
    gx_mean = abs(np.mean(gyro_data[:,0]))
    gy_mean = abs(np.mean(gyro_data[:,1]))
    gz_mean = abs(np.mean(gyro_data[:,2]))
    g_mean_list = [gx_mean, gy_mean, gz_mean]
    g_mean_list.sort() # Sorting list of numbers in ascending

    gx_range = gyro_data[np.argmax(gyro_data[:,0]), 0] - gyro_data[np.argmin(gyro_data[:,0]), 0]
    gy_range = gyro_data[np.argmax(gyro_data[:,1]), 1] - gyro_data[np.argmin(gyro_data[:,1]), 1]
    gz_range = gyro_data[np.argmax(gyro_data[:,2]), 2] - gyro_data[np.argmin(gyro_data[:,2]), 2]
    g_range_list = [gx_range, gy_range, gz_range]
    g_range_list.sort() # Sorting list of numbers in ascending
    G = g_range_list[2] 
    H = g_range_list[2]/g_range_list[1] 
    I = g_range_list[2]/g_range_list[0]


    # ----ACCELERATOR FREQUENCY DOMAIN----
    freq, Pxx = welch(acc_magnitude, fs=sampling_rate) # use of the fast Fourier transform for the estimation of power spectra
    freq_band = np.logical_and(freq >= 0.3, freq <= 15) 
    power_in_band = Pxx[freq_band] 
    freq_in_band = freq[freq_band] 
    #plt.plot(freq,Pxx)

    # D and F reflects impact of strides on acceleration
    D = np.max(power_in_band) # Feature D: the maximum energy captured by the accelerator, 
    total_power = np.sum(power_in_band) # Feature F: total power in the frequencies between 0.3 and 15 Hz:

    norm_Pxx = Pxx / total_power # normalize the power spectrum
    E = entropy(norm_Pxx) # Feature E: normalized information entropy of the discrete FFT component magnitudes

    

    sorted_idx = np.argsort(power_in_band)[::-1] 
    first_freq = freq_in_band[sorted_idx[0]] 
    second_freq = freq_in_band[sorted_idx[1]] 
    first_power = power_in_band[sorted_idx[0]] 
    second_power = power_in_band[sorted_idx[1]]

    R1 = np.sum(power_in_band[freq_in_band  < 3]) / total_power
    R3 = np.sum(Pxx[(freq >= 1.5) & (freq <= 2.5)]) / total_power 

    # ----MOVING VS: STANDING----
    moving = False
    # Append the features to the list
    if acc_mag_mean > 0.04:
        moving = True
    all_feature = [moving, acc_mag_mean ,acc_mag_std, A, B, C, Am, Bm, Cm, D, E, G, H, I, total_power, first_freq, first_power]
    return all_feature

def get_features_location_trace(trace):
    std_win = 10 # length of window in seconds
    sampling_rate = 200
    ax = get_segment_overlap(trace.data['ax'].values, sampling_rate, std_win)
    ay = get_segment_overlap(trace.data['ay'].values, sampling_rate, std_win)
    az = get_segment_overlap(trace.data['az'].values, sampling_rate, std_win)
    gx = get_segment_overlap(trace.data['gx'].values, sampling_rate, std_win)
    gy = get_segment_overlap(trace.data['gy'].values, sampling_rate, std_win)
    gz = get_segment_overlap(trace.data['gz'].values, sampling_rate, std_win)
    # add feature extraction
    num_features = 17
    num_windows = np.shape(ax)[0]
    featured_trace_location = np.zeros((num_windows, num_features))
    for i in range(num_windows):
        featured_trace_location[i,] = get_feature_location(ax[i], ay[i], az[i], gx[i], gy[i], gz[i])
    return featured_trace_location

In [7]:
# resample data to size: fixed length
def resample_data(data, fixed_length):
    num_data_points = len(data)
    x_old = np.linspace(0, 1, num_data_points)
    x_new = np.linspace(0, 1, fixed_length)
    
    # 1D interpolation for the magnitude data
    interpolator = interp1d(x_old, data, axis=0, kind='linear')
    resampled_data = interpolator(x_new)
    return resampled_data

def get_features_path_idx_trace(trace):
    resample_length = 1000
    mx = resample_data(trace.data['phone_mx'].values, resample_length)
    my = resample_data(trace.data['phone_my'].values, resample_length)
    mz = resample_data(trace.data['phone_mz'].values, resample_length)
    magn_mag = [np.sqrt(m**2+my[i]**2+mz[i]**2)for i, m in enumerate(mx)]
    return magn_mag

### load testing trace and save features of each trace

In [47]:
test_dir = 'E:\\Sunzhichao\\ETHz\\2223Spring\\Mobile_Health\\data\\test\\'

def save_testing_features(dir_traces):
    filenames = [join(dir_traces, f) for f in listdir(dir_traces) if isfile(join(dir_traces, f)) and f[-5:] == '.json']
    filenames.sort()
    features_all = {}
    for filename in filenames:
        trace_id = ''.join([*filename][-8:-5])
        trace_feature = {}
        trace = Recording(filename, no_labels=True, mute=True)
        trace_feature["activity"] = np.array(get_features_activity_trace(trace))
        trace_feature["location"] = np.array(get_features_location_trace(trace))
        trace_feature["path_idx"] = np.array(get_features_path_idx_trace(trace))
        features_all[trace_id] = trace_feature
    np.save("group00_features_test.npy", features_all)
    

In [1]:
# get and save features in test set
save_testing_features(test_dir)

## Train and save models

### Load training data

In [10]:
def segment(raw_data, all_data, data_count, std_win):
    # Calculate window size
    sampling_rate = 200
    # std_win = 3 #s
    window_size = round(std_win*sampling_rate)
    prev_data_count = len(all_data)
    for s in range(0, len(raw_data)-window_size, round(window_size/2)):
        all_data.append(raw_data[s:s+window_size])
    data_count.append(len(all_data)-prev_data_count)

# one hot encoding for activity labels
def one_hot(activity_labels):
    activitu_label_onehot = [0, 0, 0, 0]
    for i in range(4):
        if i in activity_labels:
            activitu_label_onehot[i] = 1
    return activitu_label_onehot

train_dir = 'E:\\Sunzhichao\\ETHz\\2223Spring\\Mobile_Health\\data\\train\\'
filenames = [join(train_dir, f) for f in listdir(train_dir) if isfile(join(train_dir, f)) and f!='.DS_Store']
filenames.sort()
# segmented accelerometer magnitude data for activity
magn_acc_seg = []
data_count_activity = []
# segmented accelerometer and gyroscope axis data for location
x_acc_seg = []
y_acc_seg = []
z_acc_seg = []
data_count_x_acc = []
data_count_y_acc = []
data_count_z_acc = []
x_gyro_seg = []
y_gyro_seg = []
z_gyro_seg = []
data_count_x_gyro = []
data_count_y_gyro = []
data_count_z_gyro = []

# resampled data for path index
magn_magnet_phone = []

# labels
activity_labels = []
path_labels = []
loaction_labels = []
for i, filename in enumerate(filenames):
    resample_length = 1000
    trace = Recording(filename, no_labels=False, mute=True)
    # Calculate raw magnitude of accelerometer signal
    amagn_acc = [sqrt(a**2+trace.data['ay'].values[i]**2+trace.data['az'].values[i]**2)for i, a in enumerate(trace.data['ax'].values)]
    # Pre-process data
    amagn_acc = pre_process(amagn_acc)
    segment(amagn_acc, magn_acc_seg, data_count_activity, 3)
    x_acc = pre_process(trace.data['ax'].values)
    segment(x_acc, x_acc_seg, data_count_x_acc, 10)
    y_acc = pre_process(trace.data['ay'].values)
    segment(y_acc, y_acc_seg, data_count_y_acc, 10)
    z_acc = pre_process(trace.data['az'].values)
    segment(z_acc, z_acc_seg, data_count_z_acc, 10)
    x_gyro = pre_process(trace.data['gx'].values)
    segment(x_gyro, x_gyro_seg, data_count_x_gyro, 10)
    y_gyro = pre_process(trace.data['gy'].values)
    segment(y_gyro, y_gyro_seg, data_count_y_gyro, 10)
    z_gyro = pre_process(trace.data['gz'].values)
    segment(z_gyro, z_gyro_seg, data_count_z_gyro, 10)
    # Calculate raw x, y, z of PHONE magnetometer  signal
    x_magnet_phone_resample = resample_data(trace.data['phone_mx'].values, resample_length)
    y_magnet_phone_resample = resample_data(trace.data['phone_my'].values, resample_length)
    z_magnet_phone_resample = resample_data(trace.data['phone_mz'].values, resample_length)
    magn_mag_phone = [sqrt(m**2+y_magnet_phone_resample[i]**2+z_magnet_phone_resample[i]**2)for i, m in enumerate(x_magnet_phone_resample)]
    magn_magnet_phone.append(magn_mag_phone)

    # one hot encoding
    activity_labels.append(one_hot(trace.labels['activities']))
    path_labels.append(trace.labels['path_idx'])
    loaction_labels.append(trace.labels['board_loc'])



### model for activity

In [12]:
# Load accelerometer data and labels for windows of first 50 traces
activity_labels_training = np.load('activity_labels_training.npy')
magn_data_training = np.load('activity_data_training.npy')
# add feature extraction
num_features = 25
featured_data = np.zeros((magn_data_training.shape[0], num_features))
for i in range(magn_data_training.shape[0]):
    featured_data[i,] = get_features_activity_win(magn_data_training[i])
# training 
X_train, X_test, y_train, y_test = train_test_split(featured_data, activity_labels_training, test_size = 0.2, shuffle = True)
# Compute the class weights based on the frequency of each class in the training set
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
# Set the parameters for the XGBoost model
params = {'max_depth': 5, 'eta': 0.3, 'objective': 'multi:softmax', 'num_class': 4}
# Create the XGBoost DMatrix object
dtrain = xgb.DMatrix(X_train, label=y_train)
# Train the model
xgb_model = xgb.train(params, dtrain)
print("Training XGBoost classifier for activity")
# Create the XGBoost DMatrix object for the test data
dtest = xgb.DMatrix(X_test)
# Make predictions on the test set and evaluate the model
y_pred = xgb_model.predict(dtest)
print("accuracy: ", accuracy_score(y_test, y_pred))
# Save the model to a file
# joblib.dump(xgb_model, 'activity_xgboost_model.joblib')

Training XGBoost classifier for activity
accuracy:  0.9470046082949308


### model for location

In [16]:
data_count_location = data_count_x_acc
location_labels_expand = []

for i in range(len(loaction_labels)):
    for j in range(data_count_location[i]):
        location_labels_expand.append(loaction_labels[i])
# add feature extraction
num_features = 17
num_windows = np.shape(x_acc_seg)[0]
featured_data = np.zeros((num_windows, num_features))
for i in range(num_windows):
    featured_data[i,] = get_feature_location(x_acc_seg[i], y_acc_seg[i], z_acc_seg[i], x_gyro_seg[i], y_gyro_seg[i], z_gyro_seg[i])

# train model
X_train, X_test, y_train, y_test = train_test_split(featured_data, location_labels_expand, test_size = 0.3, shuffle = True)
# Compute the class weights based on the frequency of each class in the training set
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
# Set the parameters for the XGBoost model
params = {'max_depth': 5, 'eta': 0.7, 'objective': 'multi:softmax', 'num_class': 3}
# Create the XGBoost DMatrix object
dtrain = xgb.DMatrix(X_train, label=y_train)
# Train the model
xgb_model = xgb.train(params, dtrain)
print("Training XGBoost classifier for activity")
# Create the XGBoost DMatrix object for the test data
dtest = xgb.DMatrix(X_test)
# Make predictions on the test set and evaluate the model
y_pred = xgb_model.predict(dtest)
print("accuracy: ", accuracy_score(y_test, y_pred))
# Save the model to a file
# joblib.dump(xgb_model, 'location_xgboost_model.joblib')

Training XGBoost classifier for activity
accuracy:  0.9625975217989904


### model for path index

In [17]:
# Define the CNN model for data of length 1000
class CNN_1000(nn.Module):
    def __init__(self, num_classes):
        super(CNN_1000, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        self.dropout = nn.Dropout(p=0.5)  # Add dropout layer with 50% dropout probability
        self.fc1 = nn.Linear(64 * 247, 255)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(255, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = self.dropout(x)  # Apply dropout
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x
    
# Define a custom dataset class
class PathDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
# Convert resampled data and labels to tensors
X = torch.tensor(np.expand_dims(magn_magnet_phone, axis=-1), dtype=torch.float32)
y = torch.tensor(path_labels, dtype=torch.long)
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create dataset and dataloader objects for training and validation sets
train_dataset = PathDataset(X_train, y_train)
val_dataset = PathDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the CNN model and other training components
num_classes = 5
model = CNN_1000(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=4e-06)

# Training parameters
num_epochs = 1000
best_val_accuracy = 0

# Training loop
for epoch in range(num_epochs):
    train_loss = 0.0
    model.train()
    correct_train = 0
    total_train = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        inputs = inputs.permute(0, 2, 1)
        optimizer.zero_grad()
        outputs = model(inputs)
        # Compute the number of correct predictions and the total number of predictions
        _, predicted = torch.max(outputs.data, 1)
        total_train += targets.size(0)
        correct_train += (predicted == targets).sum().item()
        # Compute the total loss, including the L1 regularization term
        loss = criterion(outputs, targets) #+ l1_regularization(model, lambda_l1)
        
        
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct_train / total_train
    
    # Validation loop
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            inputs = inputs.permute(0, 2, 1)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    val_accuracy = 100 *correct / total

    # Save the model with the best validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        model = model.to("cpu")
        # torch.save(model, "path_index_cnn_model.pt")
        model = model.to(device)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f},Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

print(f"Best validation accuracy: {best_val_accuracy:.4f}")